# Predict Single Morphology Variable (Sersic n) Based on Multiple Star-formation Variables (M*, SFR and more)


In [2]:
#Loading needed modules and classes/functions 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

#3 Linear layers NN, 1 hidden 
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)
        self.linear1 = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        x1 = self.linear1(x)
        out = self.linear(x1)
        return out




# Importing Data from Schema Table

In [3]:


data=pd.read_csv('CompleteTable.csv')  #Importing All MaNGA Data from DPRall Schema

galaxy_list=np.loadtxt('Query Results',dtype=str) #Pulling Manga ID's of galaxies which satisfy log(M) > 9 and 0 < z < 0.1

galaxy_index=np.zeros(len(galaxy_list)) 
for i in range (len(galaxy_list)): #Getting the index of these galaxies in the schema table
    galaxy_index[i]=np.where(data.loc[:,'mangaid']==galaxy_list[i])[0][0]

galaxy_index=np.array(galaxy_index,dtype=int) #Ensuring we have array that can be used to index, force int 

galaxies=data.iloc[galaxy_index] #DF of galaxies which satisfies the condition, contains all relevant schema data 

#Creating the arrays of the independent variables were are interested in, and dependent variable n 

mass=galaxies.loc[:,'nsa_sersic_mass']
log_mass=np.log10(mass)

SFR=galaxies.loc[:,'sfr_tot']
log_SFR=np.log10(SFR)

ha_flux=galaxies.loc[:,'emline_gflux_tot_ha_6564']

n=galaxies.loc[:,'nsa_sersic_n']
n=np.array(n,dtype=np.float32)
n=torch.from_numpy(n).to('cuda:0')






RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

# Prep the input data to go into a DataLoader 

In [34]:
inputs=[log_SFR,log_mass,ha_flux]

def data_preparer(inputs):  
    """
    Takes in a list in which each element is an input variable and then preps
    it accordingly to return it as one combined GPU pytorch tensor. 
    """
    for i in range (len(inputs)):
        inputs[i]=np.array(inputs[i],dtype=np.float32) #makes all inputs np arrays of np.float 32
    out=torch.from_numpy(np.column_stack(inputs)).to('cuda:0')
    return(out)

inputs_tensor=data_preparer(inputs)

#Create Tensor Dataset 
train_ds=TensorDataset(inputs_tensor,n)

#Create Data Loader
train_dl=DataLoader(train_ds,batch_size=64,shuffle=True)


